In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Mô tả bài toán


Nhiệm vụ phải làm trong bài toán là phân loại các câu insincere và sincere có trên hệ thống của Quora

# 1. Phân tích dữ liệu

In [ ]:
train_data = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_data = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")

* Như đã nêu ở trên nhiệm vụ phải làm trong bài toán là phân loại các câu insincere và sincere có trên hệ thống của Quora.
* **Input** là tập các câu hỏi tiếng anh được cho dưới dạng text và đi kèm là **id** của từng câu cũng như nhãn **label** của từng câu. 
* **output** là các giá trị Label **Sincere** (0), **Insincere** (1).

##  1.1 Tập Train
* Dữ liệu có thuộc tính qid, question_text, target
* Khi phân loại, ta dùng thuộc tính question_text là đầu vào X, target là label y

In [ ]:
train_data

In [ ]:
# Loại bỏ các thành phần dữ liệu NA trong data
train_data.dropna(inplace = True)
train_data

### Sau khi loại bỏ các dữ liệu NA chúng ta sẽ cùng xem các câu có nhãn là 1 và 0

In [ ]:
# Lấy ra câu có nhãn 1(Insincere)
toxic_data = train_data[train_data.target == 1]
## Lấy ra câu có nhãn 0(Sincere)
non_toxic_data = train_data[train_data.target == 0]

**Insincere** **Label target = 1:** 

In [ ]:
for sentence in toxic_data.question_text.sample(5):
  print(sentence)

**Sincere** **Label target = 0:** 

In [ ]:
for sentence in non_toxic_data.question_text.sample(5):
  print(sentence)

**Tỷ lệ của các câu Sincere và Insincere**

In [ ]:
import matplotlib.pyplot as plt

# Mô phỏng về độ tương quan giữa các câu insincere và sincere dưới dạng biểu đồ:

val = train_data.target.value_counts().values
names = ['Sincere', 'Insincere']
plt.bar(names, val)
plt.suptitle('Number of Sincere and Insincere Questions')
plt.show()

## Phân bố của dữ liệu

In [ ]:
labels = 'Sincere', 'Insincere'
sizes = [(non_toxic_data.shape[0] / train_data.shape[0])*100, (toxic_data.shape[0] / train_data.shape[0])*100]
plt.pie(sizes, labels=labels,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

## Nhận xét
Trong dữ liệu train, có tới **93,8%** dữ liệu **sincere** (label 0) nhưng chỉ có **6,2%** dữ liệu **Insincere** (label 1).

**Kết luận:** Mất cân bằng về dữ liệu => Độ đo sử dụng là F1-Score.
**F1_score** là trung bình điều hòa giữa precision (độ chính xác) và recall (độ bao phủ)

**Precision**: trong tập tìm được thì bao nhiêu cái (phân loại) đúng.

**Recall:** trong số các tồn tại, tìm ra được bao nhiêu cái (phân loại).

# 2. Mô tả thuật toán
## Tiền xử lý dữ liệu 
* Unicode: Chuyển về dạng unicode
* Lowercase : Chuyển về dạng in thường.
* Punctuation, Remove Number: Bỏ dấu, chữ số.
* Tokenize, Stopwords: Tách từ, Các từ dừng.
* Lemmatizers: Rút gọn từ về dạng ngắn gọn.
* => Loại bỏ các từ không ý nghĩa, giúp ta xác định được các từ cần thiết để phân loại từ đó thuộc Label 1 hay 0.

# Vector hóa dữ liệu 
* Sử dụng CountVectorizer để trích xuất các từ, biến words thành dạng vectors ở dạng Bag-of-Words bằng cách đếm số lần xuất hiện của các từ trong bộ dữ liệu.

* TF-IDF (Term Frequency – Inverse Document Frequency) là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị cao thể hiện độ quan trọng cao và nó phụ thuộc vào số lần từ xuất hiện trong văn bản nhưng bù lại bởi tần suất của từ đó trong tập dữ liệu.
 
#  Huấn luyện mô hình
* Áp dụng mô hình học máy Logistic Regression.

### 2.1. Tiền xử lý
* Sử dụng thư viện nltk(Bộ công cụ ngôn ngữ tự nhiên).

In [ ]:
import re
import nltk
import string
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

contraction_dict = {"dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not"
, "cant": "cannot", "mustnt": "must not", "ll":"will" , "re": "are" ,"ll": "will", "wont": "will not" ,"hasnt": "has not"
, "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not"
,"can't": "cannot", "‘cause": "because", "could've": "could have"
, "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not"
, "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would"
,"he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you"
, "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have"
, "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have"
, "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have"
,"i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have"
, "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us"
, "ma'am": "madam", "mayn't": "may not", "might've": "might have"
,"mightn't": "might not","mightn't've": "might not have", "must've": "must have"
, "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not"
, "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not"
, "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not"
, "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have"
, "she'll": "she will", "she'll've": "she will have", "she's": "she is"
, "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have"
, "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", 
"that'd've": "that would have", "that's": "that is", "there'd": "there would", 
"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would"
, "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
"they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
"we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
"we'll've": "we will have", "we're": "we are", "we've": "we have", 
"weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
"what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", 
"who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", 
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
"y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", 
"you'll": "you will", "youll":"you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk_stopwords = stopwords.words('english')

nltk_stopwords.remove('not')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

## Clean dữ liệu

In [ ]:
def clean(text):        
    # chuyển từ về dạng unicode 
    text = unidecode(text).encode("ascii")
    text = str(text, "ascii")

    # chuyển về dạng chữ thường, kí tự đặc biệt, chữ số.
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', ' ', text)  
    text = ''.join(i for i in text if not i.isdigit())

    # Chuyển các từ viết tắt trong từ điển về dạng thường
    tokens = word_tokenize(text)
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]
    text = " ".join(tokens)

    # Bỏ các từ chứa ở trong stop-words   
    tokens = word_tokenize(text)
    tokens_without_sw = [word for word in tokens if not word in nltk_stopwords]

    # Chuyển từ về dạng số nhiều về dạng thường
    text = [lemmatizer.lemmatize(word) for word in tokens_without_sw ] 
    text = " ".join(text)

    return text

### Dữ liệu sau khi được clean

In [ ]:
train_data['clean_questions'] = train_data['question_text'].apply(clean)
train_data

In [ ]:
X = train_data['clean_questions']
y = train_data['target']
Sincere_data = X[y == 0]
Insincere_data = X[y == 1]

In [ ]:
from collections import Counter

# top 20 words in a Sincere data
p = Counter(" ".join(Sincere_data).split()).most_common(20)
rslt = pd.DataFrame(p, columns=['Word', 'Frequency'])

rslt.plot(x='Word',kind = "barh", figsize=(12,10), title="Top 20 từ sử dụng nhiều nhất của label 0")

In [ ]:
from collections import Counter

# top 20 words in a Insincere data
p = Counter(" ".join(Sincere_data).split()).most_common(20)
rslt = pd.DataFrame(p, columns=['Word', 'Frequency'])

rslt.plot(x='Word',kind = "barh", figsize=(12,10), title="Top 20 từ sử dụng nhiều nhất của label 1")

 ## VECTOR HÓA DỮ LIỆU 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,recall_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Sử dụng thêm cách vector hóa dữ liệu bằng phương pháp TF_IDF

In [ ]:
# Khai báo các hàm thực hiện tính trọng số
count_vectorizer = CountVectorizer()
tf_idf_vectorizer = TfidfVectorizer()
# Chia tập dữ liệu đầu vào thành 2 phần (tập train và test theo tỷ lệ (7:3))
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Tiến hành tính toán trọng số của các từ trong tập huấn luyện
count_vectorizer.fit(x_train)
tf_idf_vectorizer.fit(x_train)
# Biến đổi các câu trong tập train thành ma trận trọng số cho 2 dạng vector hóa
vt_count_train = count_vectorizer.transform(x_train)
vt_count_test = count_vectorizer.transform(x_test)

vt_tfidf_train = tf_idf_vectorizer.transform(x_train)
vt_tfidf_test = tf_idf_vectorizer.transform(x_test)

 ## Huấn luyện mô hình

In [ ]:
# Khái báo mô hình
model_countVectorizer = LogisticRegression(n_jobs=10, solver='saga', C=0.1, verbose=1)
model_tfidfVectorizer = LogisticRegression(n_jobs=10, solver='saga', C=0.1, verbose=1)
# Tiến hành huấn luyện trên tập dữ liệu đã được mã hóa bằng cả 2 phương pháp
model_countVectorizer.fit(vt_count_train, y_train)
model_tfidfVectorizer.fit(vt_count_train, y_train)

y_pred_countVectorizer = model_countVectorizer.predict(vt_count_test)
y_pred_tfidfVectorizer = model_tfidfVectorizer.predict(vt_tfidf_test)

## Kết quả với Count Vectorizer

In [ ]:
print("Logistic Regression with Count Vectorizer\n")
print('Recall: ', recall_score(y_pred_countVectorizer, y_test))
print('F1 score :', f1_score(y_pred_countVectorizer, y_test), '\n')
print(classification_report(y_test, y_pred_countVectorizer))

## Kết quả với TF-IDF

In [ ]:
print("Logistic Regression with TF-IDF\n")
print('Recall: ', recall_score(y_pred_tfidfVectorizer, y_test))
print('F1 score :', f1_score(y_pred_tfidfVectorizer, y_test), '\n')
print(classification_report(y_test, y_pred_tfidfVectorizer))

**Kết luận:** Sử dụng vector hóa bằng phương pháp Count Vectorizer cho ra F1 score cao hơn nhiều so với việc sử dụng mã hóa bằng phương pháp TF_IDF.

# 3. Submission

In [ ]:
test_data['clean_questions'] = test_data['question_text'].apply(clean)
X_vec_test = count_vectorizer.transform(test_data['clean_questions'])
predictions = model_tfidfVectorizer.predict(X_vec_test)


In [ ]:
test_data['prediction'] = predictions
results = test_data[['qid', 'prediction']]
print(results)

In [ ]:
results.to_csv('submission.csv', index=False)